In [6]:
import pandas as pd
import sys
from typing import Any
import os
sys.path.append("../")
# sys.path.append("../src/")

from src.index import create_index, index_jsonl
from src.elastic.elastic_api import ElasticIndex
from src.utils import create_suggests_jsonl

In [2]:
suggest_elastic_client = ElasticIndex(index_name="suggest", 
                              elastic_host_port="8201", # Убедись что используешь правильный порт
                              elastic_password="F4ky9EiA-VfSjy6ygY6B",
                              elastic_ca_certs_path="../src/elastic/certs/http_ca.crt")

In [3]:
create_suggests_jsonl(path_to_pq="../data/yappy_hackaton_2024_40k_postprocessed.pq")

In [4]:
create_index(
        path_to_index_json="../src/elastic/settings/suggest_index.json",
        elastic_client=suggest_elastic_client,
    )

index_jsonl(path_to_jsonl="../data/suggests.jsonl", elastic_client=suggest_elastic_client)

2024-06-14 12:35:32.250 | INFO     | src.elastic.elastic_api:create_index:68 - Index with name 'suggest' is created.
2024-06-14 12:35:32.340 | INFO     | src.elastic.elastic_api:bulk_documents:170 - Indexing documents... Overall documents: 50644
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50644/50644 [00:05<00:00, 8700.84docs/s]
2024-06-14 12:35:38.217 | INFO     | src.elastic.elastic_api:count_documents_in_index:101 - Count of documents: 50644


In [62]:
def search_suggests(user_query: str, elastic_client: Any):
    """Выполнить поиск саджестов в ElasticSearch.

    :param user_query: Пользовательский запрос для подбора саджестов.
    :type user_query: str
    :param elastic_client: Клиент для взаимодействия с ElasticSearch.
    :type elastic_client: Any
    :return: Словарь с одним полем саджестов.
    :rtype: dict
    """
    body = {
            "_source": "false",
            "size":5,
            "suggest":{
               "suggest-bucket":{
                  "text": user_query,
                  "completion":{
                     "field":"suggest",
                     "size":5,
                     "skip_duplicates":"true"
                  }
               }
            }
        }

    try:
        response = elastic_client.local_client.search(index=elastic_client.index_name, body=body)["suggest"]["suggest-bucket"][0]["options"]
        completions = [doc["text"] for doc in response]
        res = {"suggests": completions}
    except Exception as e:
        # more smart exception can be here
        raise e

    return res # res

In [82]:
res = search_suggests("robl", suggest_elastic_client)

In [83]:
res

{'suggests': ['roblox', 'robloxalan', 'robloxcore', 'robloxdead', 'robloxdj']}